# Download light-curves from MACHO VS database 

TAP Query

***
__Source__ : MACHO VS multiband photometry [red and blue bands] (Alcock et al., 1996)

<br>


In [1]:
from time import strftime
from astroquery.utils.tap.core import TapPlus

In [2]:
%run ../setup_notebook

Using TensorFlow backend.


## $\ast$ TAP

In [3]:
macho_url = "http://machotap.asvo.nci.org.au/ncitap/tap"
tap       = TapPlus(url=macho_url)
#tables   = tap.load_tables()

dir_data = os.path.dirname(os.getcwd())+'/../example_data/MACHO/initial_data/'

Created TAP+ (v1.2.1) - Connection:
	Host: machotap.asvo.nci.org.au
	Use HTTPS: False
	Port: 80
	SSL Port: 443


## $\ast$ Catalog (metadata)

In [4]:
if True:
    # initial catalog contains 21474 files, 311 files are empty
    dir_catalog = f'{dir_data}/catalogs/MACHO_VS_catalog_augmented_21163sel.csv'
    catalog_meta = pd.read_csv(dir_catalog, sep = ',', header = 0, index_col=0)
    
else:
    dump_to_file = False  ## TRUE: store data
    output_file  = f'{dir_data}VS_catalog.csv' if dump_to_file else None
    
    query_catalog = "SELECT \
                    * \
                    FROM public.varstar_view \
                    ORDER BY public.varstar_view.field ASC"
    
    job = tap.launch_job_async(query_catalog,
                               dump_to_file=dump_to_file, 
                               output_file=output_file, 
                               output_format='csv')
    
    catalog_meta = job.get_results().to_pandas()
    catalog_meta.index = ['.'.join(str(int(el)) for el in row)
                for row in catalog_meta.values[:, 3:6]]

nb_vs = len(catalog_meta['starid'])
catalog_meta

,classification,type_ini,type_eb,type_lpv,period_idcatalog,period_r_ini_eb_lpv,period_b_ini_eb_lpv,period_rb_ini_eb_lpv,period_r_ini_eb,period_b_ini_eb,...,period_b_f0_lpv,amp_b_f0_lpv,period_b_f1_lpv,amp_b_f1_lpv,mag_v_lpv,mag_r_lpv,mag_w_lpv,mag_j_lpv,mag_h_lpv,mag_ks_lpv
macho_id,,,,,,,,,,,,,,,,,,,,,
1.3319.10,7,LPV_WoodB,NaN,-99.0,catalog_init_alcock01,89.22144,88.74477,88.98310,89.22144,88.74477,...,88.21,0.181,131.23,0.085,15.51,14.41,10.02,11.88,10.99,10.73
1.3441.15,4,Cepheid_Fund,NaN,NaN,catalog_init_alcock01,10.41338,10.41338,10.41338,10.41338,10.41338,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.3441.25,7,LPV_WoodB,NaN,1.0,catalog_lpv_fraser08,151.13000,151.13000,151.13000,90.36836,89.65674,...,151.13,0.909,128.95,0.708,16.33,15.19,10.60,12.61,11.61,11.35
1.3441.45,4,Cepheid_Fund,NaN,NaN,catalog_init_alcock01,3.36230,3.36224,3.36227,3.36230,3.36224,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.3441.1031,1,RR_Lyrae_AB,NaN,NaN,catalog_init_alcock01,0.50726,0.50726,0.50726,0.50726,0.50726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82.9252.185,10,Eclipsing_Binary,NaN,NaN,catalog_init_alcock01,0.61039,0.61040,0.61039,0.61039,0.61040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82.9252.186,10,Eclipsing_Binary,NaN,NaN,catalog_init_alcock01,0.51845,0.51839,0.51842,0.51845,0.51839,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82.9254.110,10,Eclipsing_Binary,NaN,NaN,catalog_init_alcock01,0.10726,0.10204,0.10465,0.10726,0.10204,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## $\ast$ Photometry

In [5]:
def tap_format_macho_data(id_vs):
    
    # Boolean True/False, if initial MACHO photometry is stored in /survey_photometry/
    # Total 21k csv files from MACHO 5Gb
    dump_to_file = True 
    
    
    m_field = str(catalog_meta['field'][id_vs])
    m_tile  = str(catalog_meta['tile'] [id_vs])
    m_seqn  = str(catalog_meta['seqn'] [id_vs])
    
    selected_name = f'{m_field}.{m_tile}.{m_seqn}'
    
    query_   = "SELECT \
                    tab_lc.fieldid , tab_lc.tileid , tab_lc.seqn, tab_lc.obsid ,\
                    tab_lc.dateobs , tab_lc.exposure , tab_lc.airmass ,\
                    tab_lc.rmag , tab_lc.rerr , tab_lc.rnormsky , tab_lc.rtype ,\
                    tab_lc.rcrowd, tab_lc.rchi2 , tab_lc.rmpix , tab_lc.rcosmicrf ,\
                    tab_lc.ramp , tab_lc.rxpix, tab_lc.rypix , tab_lc.ravesky ,\
                    tab_lc.rfwhm , tab_lc.rtobs , tab_lc.rcut ,\
                    tab_lc.bmag, tab_lc.berr , tab_lc.bnormsky , tab_lc.btype ,\
                    tab_lc.bcrowd , tab_lc.bchi2 , tab_lc.bmpix , tab_lc.bcosmicrf ,\
                    tab_lc.bamp , tab_lc.bxpix , tab_lc.bypix, tab_lc.bavesky ,\
                    tab_lc.bfwhm, tab_lc.btobs , tab_lc.bcut \
                    FROM public.varstar_view AS tab_vs \
                    INNER JOIN public.photometry_view AS tab_lc \
                    ON (tab_lc.fieldid=tab_vs.field \
                        AND tab_lc.tileid=tab_vs.tile \
                        AND tab_lc.seqn=tab_vs.seqn) \
                    WHERE (tab_lc.fieldid=" + m_field+" \
                        AND tab_lc.tileid=" + m_tile+" \
                        AND tab_lc.seqn="   + m_seqn+")"
    
    
    output_file = None
    if dump_to_file:
        output_file   = f'{dir_data}/photometry/data_TAPquery/LC_{selected_name}.csv'
    
    #job=tap.launch_job_async(query_, 
    job = tap.launch_job(query_,
                         dump_to_file=dump_to_file,
                         output_file=output_file, 
                         output_format='csv',
                         verbose=False)

    df = job.get_results().to_pandas()
    
    del job
    
    ## ----------------------------------------------------------------------- ##
    ## ----------------------------------------------------------------------- ##
    df.drop_duplicates(subset = ['dateobs'], keep = 'first', inplace = True)

    id_dateobs = df.columns.to_list().index('dateobs')  
    id_rmag    = df.columns.to_list().index('rmag')      
    id_rerr    = df.columns.to_list().index('rerr')       
    id_rcut    = df.columns.to_list().index('rcut')    
    id_bmag    = df.columns.to_list().index('bmag')     
    id_berr    = df.columns.to_list().index('berr')     
    id_bcut    = df.columns.to_list().index('bcut')    


    pix_cond_red  = ((df.values[:,id_rerr] <= 0)       
                    |(df.values[:,id_rerr] == 9.999)       # unreliable 
                    |(df.values[:,id_rmag] == 99.999)      # unreliable
                    |(df.values[:,id_rerr] >= 29.999)      # "29.999" = below threshold
                    |(df.values[:,id_rmag] >= 29.999)    
                    |(df.values[:,id_rcut] == 0) )
    pix_cond_red  = np.where(pix_cond_red)[0]

    pix_cond_blue = ((df.values[:,id_berr] <= 0) 
                    |(df.values[:,id_berr] == 9.999)      # unreliable 
                    |(df.values[:,id_bmag] == 99.999)     # unreliable 
                    |(df.values[:,id_berr] >= 29.999)     # "29.999" = below threshold
                    |(df.values[:,id_bmag] >= 29.999)  
                    |(df.values[:,id_bcut] == 0))
    pix_cond_blue = np.where(pix_cond_blue)[0]


    ## ----------------------------------------------------------------------- ##
    ## TIME SERIES
    ## ----------------------------------------------------------------------- ##
    selected_period = catalog_meta.period_b_ini_eb.loc[selected_name]  ## Xmatch initial_MACHO & Derekas07
    
    if selected_name=='53.3849.67':
        selected_period = 170.5658855585837  #initially 177.89282 in MACHO_catg (detectable periodicity for P=170 days)
    
    
    lc_red=None; lc_blue=None; lc_rb_norm=None;
    ## ---------------- R-BAND ---------------- ## 
    df_red  = copy.deepcopy(df)
    df_red  = df_red.drop(pix_cond_red)  
    inds    = np.argsort(df_red['dateobs'])
    lc_red  = LightCurve(name=selected_name, survey  = 'MACHO',
                         times          = df_red['dateobs'].values[inds],
                         measurements   = df_red['rmag'].values[inds],
                         errors         = df_red['rerr'].values[inds],
                         period_catalog = selected_period
                        )
    lc_red.cut       = (df_red['rcut'].values[inds]).astype(int)
    lc_red.label     = LC_types[catalog_meta.classification.loc[selected_name]]
    lc_red.passbands = np.full(len(lc_red.times), dict_filters['red'])
    lc_red.evaluate_trend_spline()                ## trend_spline
    lc_red.evaluate_trend_linear()                ## trend_linear
    lc_red.fit_supersmoother(periodic=True)  ## SuperSmoother residual 
    mags_r_norm, errs_r_norm, m_stats = m_preprocess.normalize_lc(lc_red.measurements, lc_red.errors)
    lc_red.stats     = m_stats
    lc_red.mean      = m_stats['mean']
    lc_red.scale     = m_stats['std']
    lc_red.infos     = "r-band_timeserie"
    lc_red.nb_points = len(lc_red.times)
    
    
    ## ---------------- B-BAND ---------------- ## 
    df_blue = copy.deepcopy(df)
    df_blue = df_blue.drop(pix_cond_blue)  
    inds    = np.argsort(df_blue['dateobs'])
    lc_blue = LightCurve(name=selected_name, survey='MACHO', label=lc_red.label,
                         times          = df_blue['dateobs'].values[inds],
                         measurements   = df_blue['bmag'].values[inds],
                         errors         = df_blue['berr'].values[inds],
                         period_catalog = selected_period
                        )
    lc_blue.cut       = (df_blue['bcut'].values[inds]).astype(int)
    lc_blue.passbands = np.full(len(lc_blue.times), dict_filters['blue'])
    lc_blue.evaluate_trend_spline()               ## trend_spline
    lc_blue.evaluate_trend_linear()               ## trend_linear
    lc_blue.fit_supersmoother(periodic=True)  ## SuperSmoother residual
    mags_b_norm, errs_b_norm, m_stats = m_preprocess.normalize_lc(lc_blue.measurements, lc_blue.errors)
    lc_blue.stats     = m_stats
    lc_blue.mean      = m_stats['mean']
    lc_blue.scale     = m_stats['std']
    lc_blue.infos     = "b-band_timeserie"
    lc_blue.nb_points = len(lc_blue.times)

    
    
    ## ---------------- RB ---------------- ##                               
    lc_time = np.r_[lc_red.times, lc_blue.times]
    inds = np.argsort(lc_time)
    lc_rb_norm = LightCurve(name=lc_red.name, survey='MACHO', label=lc_red.label,
                               times        = lc_time[inds], 
                               measurements = (np.r_[mags_r_norm, mags_b_norm])[inds], 
                               errors       = (np.r_[errs_r_norm, errs_b_norm])[inds],
                               passbands    = (np.r_[lc_red.passbands, lc_blue.passbands])[inds],
                               period_catalog = selected_period )
    mags_rb_norm, errs_rb_norm, m_stats = m_preprocess.normalize_lc(lc_rb_norm.measurements, lc_rb_norm.errors)
    lc_rb_norm.stats     = m_stats
    lc_rb_norm.mean      = m_stats['mean']
    lc_rb_norm.scale     = m_stats['std']
    #lc_rb_norm.evaluate_trend_spline()         ## trend_spline
    #lc_rb_norm.evaluate_trend_linear()         ## trend_linear
    lc_rb_norm.fit_supersmoother(periodic=True, m_period=selected_period) 
    lc_rb_norm.infos = ['rb-band_timeserie_norm']
    lc_rb_norm.nb_points = len(lc_rb_norm.times)
    
    
    ## ----------------------------------------------------------------------- ##
    ## STORE
    ## ----------------------------------------------------------------------- ##
    dict_results = {'name'      : selected_name, 
                    'period'    : selected_period, 
                    'label'     : lc_red.label,
                    'nbpoints'  : {'red' : len(lc_red.times), 
                                   'blue': len(lc_blue.times), 
                                   'rb':   len(lc_rb_norm.times)},
                    'lc_red'    : lc_red,     
                    'lc_blue'   : lc_blue,     
                    'lc_rb_norm': lc_rb_norm,  
                   }
    
    #sleep(.5)
    return dict_results
    

In [ ]:
%%time

logfilename = strftime(f'{dir_data}logs/log_TAP_MACHO_%Y-%m-%d.log')
m_logs = open(logfilename,'w') 
m_logs.write('#'*40+'\n')
m_logs.write(strftime('# LOG (TAP+QUERY, MACHO VS database) : %Y-%m-%d %H:%M:%S\n')) 
m_logs.write('#'*40+'\n')

## -------------------------------------------------------- ##
nprocesses = 75 #8

m_localrun = 1

nstep = 100; d = range(400) #nb_vs)
list_runs = [d[x:x+nstep] for x in range(0,len(d),nstep)]

res_list = []
for i in range(len(list_runs)):
    print(f"{(i+1)}, {list_runs[i][0]:5d}/{nb_vs} LC, ", datetime.datetime.now())

    pool = None
    list_vs  = list_runs[i]
    
    ## ################################ ##
    if m_localrun==0: 
        import schwimmbad
        with schwimmbad.MultiPool(nprocesses) as pool:  
            results = pool.imap(tap_format_macho_data, list_vs)  
            for res in results:
                res_list.append(res)
                ##pool.wait_timeout()
        pool.close()
        pool.join()
        
    ## ################################ ##
    else :  
        import multiprocessing
        with multiprocessing.Pool(nprocesses) as pool:
            results = pool.map_async(tap_format_macho_data, list_vs)
            results.wait()
            res_list.extend(results.get())
        pool.close()
        pool.join()

    
## -------------------------------------------------------- ##
for res in res_list:
    m_logs.write(">>\t MACHO_ID:"+res['name']+"\t\t#datapoints:"+str(res['nbpoints'])+"\n")
m_logs.write(">>>>>> \tEND JOB "+ strftime('%Y-%m-%d %H:%M:%S\n')) 
m_logs.close() 


In [15]:
%%time

list_names   = [res['name']   for res in res_list]
list_periods = [res['period'] for res in res_list]
list_labels  = [res['label']  for res in res_list]

list_lc_red           = [res['lc_red']     for res in res_list]
list_lc_blue          = [res['lc_blue']    for res in res_list]
list_lc_rb_norm       = [res['lc_rb_norm'] for res in res_list]


## -------------------------------------------------------- ##
dir_store = f'{dir_data}/photometry/'

if os.path.exists(f'{dir_store}timeseries')==False:
    os.mkdir(f'{dir_store}timeseries')

joblib.dump(list_lc_red,     f'{dir_store}timeseries/lcs_red.pkl',     compress=5)
joblib.dump(list_lc_blue,    f'{dir_store}timeseries/lcs_blue.pkl',    compress=5)
joblib.dump(list_lc_rb_norm, f'{dir_store}timeseries/lcs_rb_norm.pkl', compress=5)

del list_names, list_periods, list_labels, list_lc_red, list_lc_blue, list_lc_rb_norm, 


CPU times: user 1.08 ms, sys: 2.44 ms, total: 3.52 ms
Wall time: 3.45 ms


In [ ]:
'''
%%time

logfilename = strftime(dir_data+'log_TAP_MACHO_%Y-%m-%d.log')
logfilepath = "%s/%s" % (os.getcwd(),logfilename) 

m_logs = open(logfilepath,'w') 
m_logs.write('#'*40+'\n')
m_logs.write(strftime('# LOG (TAP+ QUERY, MACHO VS database) : %Y-%m-%d %H:%M:%S\n')) 
m_logs.write('#'*40+'\n')

for id_vs in range(50): #range(nb_vs): ## 21474 objects in database (311 empty tables)
    
    if ((id_vs%(nb_vs//10))==0)|(id_vs==nb_vs-1):
        print(f"{(id_vs+1):5d}/{nb_vs} LC, ", strftime('%Y-%m-%d %H:%M:%S\n'))
            
    m_field = str(resu_vs['field'][id_vs])
    m_tile  = str(resu_vs['tile'][id_vs])
    m_seqn  = str(resu_vs['seqn'][id_vs])
    
    query_   = "SELECT \
                    tab_lc.fieldid , tab_lc.tileid , tab_lc.seqn, tab_lc.obsid ,\
                    tab_lc.dateobs , tab_lc.exposure , tab_lc.airmass ,\
                    tab_lc.rmag , tab_lc.rerr , tab_lc.rnormsky , tab_lc.rtype ,\
                    tab_lc.rcrowd, tab_lc.rchi2 , tab_lc.rmpix , tab_lc.rcosmicrf ,\
                    tab_lc.ramp , tab_lc.rxpix, tab_lc.rypix , tab_lc.ravesky ,\
                    tab_lc.rfwhm , tab_lc.rtobs , tab_lc.rcut ,\
                    tab_lc.bmag, tab_lc.berr , tab_lc.bnormsky , tab_lc.btype ,\
                    tab_lc.bcrowd , tab_lc.bchi2 , tab_lc.bmpix , tab_lc.bcosmicrf ,\
                    tab_lc.bamp , tab_lc.bxpix , tab_lc.bypix, tab_lc.bavesky ,\
                    tab_lc.bfwhm, tab_lc.btobs , tab_lc.bcut \
                    FROM public.varstar_view AS tab_vs \
                    INNER JOIN public.photometry_view AS tab_lc \
                    ON (tab_lc.fieldid=tab_vs.field \
                        AND tab_lc.tileid=tab_vs.tile \
                        AND tab_lc.seqn=tab_vs.seqn) \
                    WHERE (tab_lc.fieldid=" + m_field+" \
                        AND tab_lc.tileid=" + m_tile+" \
                        AND tab_lc.seqn="   + m_seqn+")"
    
    selected_name = f'{m_field}.{m_tile}.{m_seqn}'
    output_file = f'{dir_data}photometry/LC_{selected_name}.csv'
        
    #job=tap.launch_job_async(query_, 
    job = tap.launch_job(query_,
                         dump_to_file=True,
                         output_file=output_file, 
                         output_format='csv',
                         verbose=False)

    resu = job.get_results()
    
    m_logs.write(f">>>>>> \tMACHO ID (field.tile.seqn): {selected_name}\n")
    m_logs.write(f">>>>>> \t#LC_datapoints {len(resu)}\n")
    m_logs.write('-'*40+'\n')
                                
m_logs.write(">>>>>> \tEND JOB "+ strftime('%Y-%m-%d %H:%M:%S\n')) 
m_logs.close() 


print()
'''